# pattern court decisions

In [1]:
#

In [10]:
import pandas
import os
import requests
import re

In [12]:
#define the website string
websites = ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:62021TA0582','https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:62021TA0426']
content = []
for site in websites:
    response = requests.get(site)
    print(response.status_code)
    data = response.text
    content.append(data)
    
    

200
200


In [13]:
content

['<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<!-- CONVEX # converter_version:9.8.0 # generated_on:20230721-0911 -->\r\n<head><meta name="format-detection" content="telephone=no"/>\n   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n   <script type="text/javascript" src="/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=1e2d85f6ff8ee95d|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1720184838465|tp=500,50,0|agentUri=/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/eurlex-frontoffice/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_-1570085090|rpid=1149823617|domain=europa.eu"></script><link type="text/css" rel="stylesheet" href="./../../../../css/oj/oj-convex.css"/>\n   <title>C_2023252EN.01004201

In [14]:
name_pattern = r'Orders (.+?) to pay'
#for dismissal entity name

In [15]:
# Create patterns
delist_pattern = r'Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern2 = r'in so far as they concern (.+?);2.Orders'
name_pattern1 = r'Orders (.+?) to pay'

# Create a list to store all court decisions
court_decisions = []

# Iterate over the content
for i in content:
    # Get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    
    # Initialize variables
    Delisting = None
    delisting_case = None
    entity_name = None
    
    # Check for delisting
    if re.search(delist_pattern, text):
        Delisting = 1
    else:
        Delisting = 0
    if Delisting == 1:
        match = re.search(name_pattern2, text)
        if match:
            entity_name = match.group(1)
    if Delisting == 0:
        match = re.search(name_pattern1, text)
        if match:
            entity_name = match.group(1)
    
    # Extract delisting case
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)

    # Create dictionary for current court decision
    court_decision = {
        'entity_name': entity_name,
        'Delisting': Delisting,
        'Delisting_case': delisting_case,
        'url': re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data),
        'celex' : re.findall(r'uri=CELEX%([^"]+)', data),
        'publication_date': re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data),
        'date': re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data),
        'title': re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data),
        'law_number': re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)</p>', data),
        'case_number': re.findall(r'Case\sT-\d+/\d+', data),
        'legal_base': re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
    }
    
    # Append the current decision dictionary to the list
    court_decisions.append(court_decision)

# Print the list of dictionaries
print(court_decisions, encoding = 'UTF-8') #encoding ändern oder string subsitute (string punkt replace xa0 mit .)

[{'entity_name': 'Mr\xa0Siarhei Rubnikovich', 'Delisting': 0, 'Delisting_case': None, 'url': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A62021TA0426'], 'celex': ['3A62021TA0426'], 'publication_date': ['2.5.2023'], 'date': [], 'title': ['Judgment of the General Court of 8\xa0March 2023\xa0— Assaad v Council'], 'law_number': ['155/50'], 'case_number': ['Case T-426/21'], 'legal_base': [('CFSP', '2021/751'), ('EU', '2021/743'), ('CFSP', '2022/849'), ('EU', '2022/840'), ('CFSP', '2021/751'), ('EU', '2021/743'), ('CFSP', '2022/849'), ('EU', '2022/840')]}, {'entity_name': 'Mr\xa0Nizar Assaad', 'Delisting': 1, 'Delisting_case': 'Council Implementing Decision (CFSP) 2021/751 of 6\xa0May 2021 implementing Decision 2013/255/CFSP concerning the restrictive measures against Syria, Council Implementing Regulation (EU) 2021/743 of 6\xa0May 2021 implementing Regulation (EU) No\xa036/2012 concerning restrictive measures in view of the situation in Syria, Council Decision (CFSP) 2

In [9]:
court_decisions

[{'entity_name': 'Mr\xa0Siarhei Rubnikovich',
  'Delisting': 0,
  'Delisting_case': None,
  'url': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A62021TA0426'],
  'publication_date': ['2.5.2023'],
  'date': [],
  'title': ['Judgment of the General Court of 8\xa0March 2023\xa0— Assaad v Council'],
  'law_number': ['155/50'],
  'case_number': ['Case T-426/21'],
  'legal_base': [('CFSP', '2021/751'),
   ('EU', '2021/743'),
   ('CFSP', '2022/849'),
   ('EU', '2022/840'),
   ('CFSP', '2021/751'),
   ('EU', '2021/743'),
   ('CFSP', '2022/849'),
   ('EU', '2022/840')]},
 {'entity_name': 'Mr\xa0Nizar Assaad',
  'Delisting': 1,
  'Delisting_case': 'Council Implementing Decision (CFSP) 2021/751 of 6\xa0May 2021 implementing Decision 2013/255/CFSP concerning the restrictive measures against Syria, Council Implementing Regulation (EU) 2021/743 of 6\xa0May 2021 implementing Regulation (EU) No\xa036/2012 concerning restrictive measures in view of the situation in Syria, Council D

In [8]:
stop

NameError: name 'stop' is not defined

In [ ]:
#now extract th information
# Initialize variables
Delisting = None
delisting_case = None
entity_name = None
#create patterns
delist_pattern = r'The Court: 1. Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern = r'in so far as they concern (.+?);2.Orders'
#create empty dic
results = {}

for i in content:
    #get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    if re.search(delist_pattern, text):
        Delisting = 0
        print('Dismissal')
    else:
        Delisting = 1  # Assuming 1 for non-dismissal cases
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)
    # Extract entity name
    match = re.search(name_pattern, text)
    if match:
        entity_name = match.group(1)
    

In [ ]:
#now get it into a dictionary format
court_decision = {}
court_decision['entity_name'] = entity_name
court_decision['Delisting'] = Delisting
court_decision['Delisting_case']= delisting_case
court_decision['url'] = re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
court_decision['publication_date'] =re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
court_decision['date'] =  re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data)
court_decision['title']= re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data)
court_decision['law_number'] = re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)',data)
court_decision['case_number'] = re.findall(r'Case\sT-\d+/\d+', data)
court_decision['legal_base'] = re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
